In [12]:
from gemlib.database.utils import load_data_oracle, load_data_hub
import pandas as pd
# import warnings to avoid getting sqlalchemy warnings from load_data_hub
import warnings
warnings.filterwarnings('ignore')

In [13]:
zones = ("22Bolo", "24Ama1", "26Fanh", "27Arr1", "29Sob2", "44Milagre", "48Vale_de", "60Maravil", "61MaraviII", "68Jogui")


In [14]:
import nbimporter
dh_qry = """
	SELECT * 
	FROM raw_tdmi.lov_edpr_facilities
"""

df_fac_props = load_data_hub(dh_qry)

In [15]:

df_fac_props['facility_name'] = df_fac_props['FACILITY_UNIT_NAME'].apply(lambda x: x.split(' -')[0] if ' -' in x else x)

df_fac_props['FACILITY_UNIT_ID_SHORT'] = df_fac_props['FACILITY_UNIT_ID'].apply(lambda x: '_'.join(x.split('_')[1:]) if '_' in x else x)

df_zones_props = df_fac_props[df_fac_props['FACILITY_UNIT_ID_SHORT'].isin(zones)]

In [16]:
orc_qry = f"""
	SELECT * 
	FROM POWERBI_EOL_PREV_HIST
	WHERE NOME_MODELO IN {zones}
-- 	WHERE NOME_MODELO == "22Bolo"
"""


In [17]:

df = load_data_oracle(orc_qry)

In [18]:
df_index = df.copy()

In [19]:
# filter datetime to get the last 2 years
hist_num_year = 4

df_index = df_index[df_index['dia_aplicacao'] >= df_index['dia_aplicacao'].max() - pd.DateOffset(months=hist_num_year*12)]


In [20]:
# create datetimeindex based on dia_aplicacao(datetime) e hora_aplicacao(float)

df_index['datetime'] = pd.to_datetime(df_index['dia_aplicacao']) + pd.to_timedelta(df_index['hora_aplicacao'] - 1, unit='h')
df_index.set_index('datetime', inplace=True)
df_index.drop(columns=['dia_aplicacao', 'hora_aplicacao', 'zonadescr', 'unidade', 'percentil', 'nomezona', 'modelo_descr'], inplace=True)
df_index.sort_index(inplace=True)

In [21]:
#calculate the mean of the duplicates, keeping the original index and nome_modelo eliminating the duplicates reset the nome_modelo index
df_silver = (
	df_index
	.groupby(['datetime', 'nome_modelo'])
	.mean()
	.reset_index(level='nome_modelo')
)

df_silver['installed_capacity'] = (df_silver['nome_modelo'].map(df_zones_props.set_index('FACILITY_UNIT_ID_SHORT')['INSTALLED_CAPACITY']))/1000

df_silver['capacity_factor'] = df_silver['valor'] / df_silver['installed_capacity']

df_silver.drop(columns=['installed_capacity', 'valor'], inplace=True)



In [22]:
df_silver_pivot = df_silver.pivot(columns='nome_modelo', values='capacity_factor')

In [23]:
df_silver.to_csv('df_silver.csv')